In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import regex as re

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 117.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.1 MB/s eta 0:00:00


In [ ]:
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

In [ ]:
bert_model_name = "sagorsarker/bangla-bert-base"
bert = BertModel.from_pretrained(bert_model_name)
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [ ]:
class NewsBert(nn.Module):

    def __init__(self, bert):
        super(NewsBert, self).__init__()

        self.bert = bert

        # dropout layer
        self.dropout = nn.Dropout(0.2)

        # relu activation function
        self.relu = nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768, 128)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(128, 2)

    # define the forward pass
    def forward(self, input_ids, token_type_ids, attention_mask):
        # pass the inputs to the model
        out = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        x = self.fc1(out[1])
        x = self.relu(x)
        # output layer
        x = self.fc2(self.dropout(x))

        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NewsBert(bert)
model.to(device);

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Dataset/news_model2.pth", map_location = device))

<All keys matched successfully>

In [ ]:
def classify_real_life_text(user_text):
    # Tokenize the user's text
    inputs = tokenizer(user_text, padding=True, truncation=True, return_tensors="pt")

    # Move input tensors to the appropriate device
    input_ids = inputs["input_ids"].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Make predictions
    with torch.no_grad():
        otp = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

    # Get the predicted class (0 or 1 in binary classification)
    preds = otp.detach().cpu().numpy()
    preds = np.argmax(preds, axis = 1)

    return preds

In [ ]:
# Example of using the function with real-life text
user_text = input("Enter a text for classification: ")
predicted_class = classify_real_life_text(user_text)
if(predicted_class==0):
  print("it is true")
else:
  print("false")

Enter a text for classification: দেশে নিষিদ্ধ হচ্ছে ফ্রি ফায়ার ও পাবজি গেম
false


In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=731490c4af0609f28c361cfc714176db894393907d97e0f2092f4bfe44000727
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [ ]:
port_no = 5002

In [ ]:
import os
import torch
import numpy as np
from flask import Flask, render_template, request



# Step 1: Create the "templates" folder
if not os.path.exists("templates"):
    os.mkdir("templates")


In [ ]:
from flask import Flask, render_template, request
app = Flask(__name__, template_folder='templates')
ngrok.set_auth_token("2X7sDlJuX1cfi0wiHOCxiHXidwf_3j4WdHEQDKxSCiBeW8vh4")
public_url =  ngrok.connect(port_no).public_url

@app.route("/", methods=["GET", "POST"])
def home():
    if request.method == "POST":
        question = request.form.get("question")
        predictions = classify_real_life_text(question)
        if predictions[0] == 0:
            result = "True"
        else:
            result = "False"
        return render_template('answer.html', question=question, result=result)
    return render_template('answer.html')

print(f"To acces the Gloable link please click {public_url}")

app.run(port=port_no)

To acces the Gloable link please click https://93da-34-138-61-108.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5002
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Nov/2023 08:45:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Nov/2023 08:45:32] "GET /favicon.ico HTTP/1.1" 404 -
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
INFO:werkzeug:127.0.0.1 - - [07/Nov/2023 08:46:02] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Nov/2023 08:46:30] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Nov/2023 08:46:45] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Nov/2023 08:46:59] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Nov/2023 08:47:14] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Nov/2023 08:47:37] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Nov/2023 08:48

In [ ]:
ngrok.set_auth_token("2X7sDlJuX1cfi0wiHOCxiHXidwf_3j4WdHEQDKxSCiBeW8vh4")